# Bedrock Stable Diffusion

---

Amazon Bedrock에서 Stability AI의 Text-to-Image 모델을 사용할 수 있습니다. (24년 9월 4일)

Stable Image Ultra, Stable Diffusion 3 Large, Stable Image Core 모델은 모델은 다중 주제 프롬프트, 이미지 품질 및 타이포그래피에서 성능을 크게 개선하며 마케팅, 광고, 미디어, 엔터테인먼트, 리테일 등 다양한 사용 사례에 대한 고품질 비주얼을 빠르게 생성하는 데 사용할 수 있습니다. ([링크](https://aws.amazon.com/blogs/aws/stability-ais-best-image-generating-models-now-in-amazon-bedrock/))

Stable Diffusion XL(SDXL)에 비해 Stable Image Ultra와 Stable Diffusion 3 Large의 주요 개선 사항 중 하나는 생성된 이미지의 텍스트 품질이 향상되고 혁신적인 Diffusion Transformer 아키텍처 덕분에 철자 및 인쇄 오류가 줄어든다는 것입니다. 이 아키텍처는 이미지와 텍스트에 대해 두 가지 별도의 가중치 세트를 구현하지만 두 모달리티 간의 정보 흐름을 가능하게 합니다.

### Request Field

- `mode` (str)
  - `text-to-image`, `image-to-image` (only use in Stable Image 3)
- `aspect_ration` (str)
  - `16:9` / `1:1` / `21:9` / `2:3` / `3:2` / `4:5` / `5:4` / `9:16` / `9:21`
- `negative_prompt`
- `seed`
- `output_format` (str)
  - `png` / `jpeg` / `webp`

### 참고 링크 

- [stability.ai API 문서](https://platform.stability.ai/docs/api-reference#tag/Generate/paths/~1v2beta~1stable-image~1generate~1ultra/post)
- [Bedrock Stability AI 문서](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-stability-diffusion.html)

---

In [ ]:
!pip install --quiet boto3 langchain

In [ ]:
import os
import sys

ROOT_PATH = os.path.abspath("../")
sys.path.append(ROOT_PATH)

import re
import json
import boto3
from typing import List
from PIL import Image, ImageOps
from langchain.prompts import PromptTemplate

from common.aws.claude import BedrockClaude
from common.aws.titan_image import BedrockTitanImage, ImageParams, ImageSize
from common.utils.images import display_image, encode_image_base64_from_file, save_base64_image


def invoke_sd_model(modelId, request: dict):
    bedrock = boto3.client("bedrock-runtime")
    try:
        response = bedrock.invoke_model(body=json.dumps(request), modelId=modelId)
        response_body = json.loads(response.get("body").read())
        return response_body["images"][0]
    except Exception as e:
        print(f"Cannot generate a image: {e}")
        return []

In [ ]:
MODEL_STABLE_IMAGE_ULTRA = "stability.stable-image-ultra-v1:0"
MODEL_STABLE_IMAGE_CORE = "stability.stable-image-core-v1:0"
MODEL_SD3_LARGE = "stability.sd3-large-v1:0"
MODEL_SDXL = "stability.stable-diffusion-xl-v1"

bedrock_runtime = boto3.client("bedrock-runtime")
prompt = "a beautiful mountain landscape"
seed = 44
aspect_ratio = "16:9"
output_format = "png"

OUTPUT_PATH = "./data"

### Stable Image Ultra

---

최고 품질의 사실적인 결과물을 생성하므로 전문 인쇄 미디어 및 대형 포맷 용도에 적합합니다. Stable Image Ultra는 뛰어난 디테일과 사실감을 렌더링하는 데 탁월합니다.

---


In [ ]:
body = {
    "prompt": prompt,
    "mode": "text-to-image",
    "aspect_ratio": aspect_ratio,
    "seed": seed,
    "output_format": output_format

}

base64_image_1 = invoke_sd_model(MODEL_STABLE_IMAGE_ULTRA, body)
display_image(base64_image_1)
save_base64_image(base64str=base64_image_1, path=f"{OUTPUT_PATH}/image-1.{output_format}")

### Stable Image Core

---

빠르고 경제적인 이미지 생성에 최적화되어 아이디어 구상 중 콘셉트를 빠르게 반복하는 데 적합합니다.

---


In [ ]:
body = {
    "prompt": prompt,
    "mode": "text-to-image",
    "aspect_ratio": aspect_ratio,
    "seed": seed,
    "output_format": output_format
}

base64_image_2 = invoke_sd_model(MODEL_STABLE_IMAGE_CORE, body)
display_image(base64_image_2)
save_base64_image(base64str=base64_image_2, path=f"{OUTPUT_PATH}/image-2.{output_format}")

### Stable Diffusion 3 Large

---

생성 속도와 출력 품질 간의 균형을 유지합니다. 웹 사이트, 뉴스레터, 마케팅 자료 등 대량의 고품질 디지털 자산을 제작하는 데 이상적입니다.

---


In [ ]:
# Text-to-Image
body = {
    "prompt": prompt,
    "mode": "text-to-image",
    "aspect_ratio": aspect_ratio,
    "seed": seed,
    "output_format": output_format

}

base64_image_3 = invoke_sd_model(MODEL_SD3_LARGE, body)
display_image(base64_image_3)
save_base64_image(base64str=base64_image_3, path=f"{OUTPUT_PATH}/image-3.{output_format}")

In [ ]:
# Image-to-Image
change_prompt = "wide lake, cloudy sky, art"
body = {
    "prompt": change_prompt,
    "mode": "image-to-image",
    "image": base64_image_1,
    "strength": 0.7,
    "seed": seed,
    "output_format": output_format

}

base64_image_4 = invoke_sd_model(MODEL_SD3_LARGE, body)
display_image(base64_image_4)
save_base64_image(base64str=base64_image_4, path=f"{OUTPUT_PATH}/image-4.{output_format}")

# Multimodal Prompt-to-Image Generation

---

이미지 생성 모델을 사용하여 원하는 이미지를 생성하려면 프롬프트에서 이미지의 세부 사항을 정확하게 설명하는 것이 중요합니다. 그러나 사전 정보 없이 자세한 프롬프트를 만드는 것은 어려울 수 있습니다. 이를 돕기 위해 Multimodal LLM을 통해 사용자가 제공하는 스타일과 키워드를 기반으로 한 창의적인 이미지 프롬프트 아이디어를 제안합니다. 이를 통해 사용자는 보다 직관적으로 이미지 프롬프트를 생성할 수 있으며, 프롬프트 작성에 소요되는 노력을 줄일 수 있습니다.

---

In [ ]:
boto3_session = boto3.session.Session()
print(f'The notebook will use aws services hosted in {boto3_session.region_name} region')

image_params = ImageParams()
image_params.set_configuration(count=1, size=ImageSize.SIZE_1152x640)
titanImageGen = BedrockTitanImage(
    region=boto3_session.region_name,
    modelId='amazon.titan-image-generator-v2:0'
)

## Basic Prompt

---

기본 이미지 생성 프롬프트를 사용했을 때 입니다. Titan Image Generation 모델은 Input Text를 영어만 지원합니다.

---

In [ ]:
keyword="champagne",
style="realistic"

# titan g1 v2
body = image_params.text_to_image(text=f"{keyword}, {style}")
img = titanImageGen.generate_image(body)[0]
display_image(img)
save_base64_image(img, f"{OUTPUT_PATH}/gen_titan_basic.png")

# stable diffusion
img = invoke_sd_model(MODEL_SD3_LARGE, {
    "prompt": f"{keyword}, {style}",
    "aspect_ratio": aspect_ratio,
    "seed": seed,
    "output_format": output_format
})

display_image(img)
save_base64_image(img, f"{OUTPUT_PATH}/gen_sd_basic.png")

## LLM Prompt

---
LLM으로부터 Image Generation을 위한 프롬프트를 생성한 결과입니다. `style`, `keyword` 값을 입력하면, 해당 내용을 바탕으로 3개의 이미지 프롬프트를 제안합니다.

- `style`: 원하는 무드나 스타일
- `keyword`: 생성하고자 하는 이미지의 키워드
---

In [ ]:
PROMPT = """You are an Assistant that creates prompts for generate background image by image generator model. The image that Human wants is written in <keyword>.
Follow style guide in <style> and write three prompts for creating image keeping it to 300 characters or less. Use this fomat without further explanation: 
<prompt>image prompt</prompt>

<keyword>
{keyword}
</keyword>

<style>
{style}
</style>
"""

def extract_format(result_string):
    pattern = r'<prompt>(.*?)</prompt>'
    return re.findall(pattern, result_string)

def get_prompt(keyword: str, style: str): 
    return PromptTemplate(
                template=PROMPT,
                input_variables=["keyword", "style"]
            ).format(keyword=keyword,
                     style=style)

def get_prompt_by_llm(prompt: str) -> List[str]:
    claude = BedrockClaude()
    res = claude.invoke_llm_response(prompt)
    return extract_format(res)

In [ ]:
prompt = get_prompt(
    keyword=keyword,
    style=style
)

image_prompts = get_prompt_by_llm(prompt)
print(image_prompts)

In [ ]:
for idx, prompt in enumerate(image_prompts):
    print(f"[PROMPT] {prompt}")

    # titan g1 v2
    body = image_params.text_to_image(text=prompt)
    img = titanImageGen.generate_image(body)[0]

    display_image(img)
    save_base64_image(img, f"{OUTPUT_PATH}/gen_titan_llm_{idx+1}.png")

    # stable diffusion
    img = invoke_sd_model(MODEL_SD3_LARGE, {
        "prompt": prompt,
        "aspect_ratio": aspect_ratio,
        "seed": seed,
        "output_format": output_format
    })
    display_image(img)
    save_base64_image(img, f"{OUTPUT_PATH}/gen_sd_llm_{idx+1}.png")

## Multimodal LLM Prompt

---

Titan Image Generator 는 레퍼런스 이미지를 참조하여 새로운 이미지를 생성하는 몇 가지 옵션을 제공합니다.

- **Image Conditioning**: 텍스트 프롬프트를 따르면서 참조 이미지의 레이아웃 및 구성과 일치하는 이미지를 생성
  - 단순히 기존 이미지를 재구성하는 데 그칠 수 있어, 창의적인 자유도가 제한
- **Color Guided Generation**: `referenceImage`를 통해 참조 색상들을 받아와 이미지의 색상 톤을 반영
  - 일부 색상 톤을 반영하지만 이미지의 분위기나 재질 등을 반영하기 어려움
- **Image Variation**: 소스 이미지는 보존하고, 스타일과 배경을 변형한 이미지 생성
  - 변형의 범위가 제한적이고 원본 이미지에 제약이 있음

위 방법들은 레퍼런스 이미지의 특정 속성(색상, 레이아웃, 소스 이미지 등)을 반영하는 데에는 유용하지만, 변형의 범위가 제한적이다 보니 창의적이고 복합적인 특징을 반영한, 완전히 새로운 이미지를 생성하는 데에 한계가 있습니다. Multimodal LLM을 통해 레퍼런스 이미지를 이해하고, 이를 바탕으로 타겟하는 이미지를 생성하도록 프롬프트를 생성하면 더 자유롭고 창의적인 이미지 프롬프트 생성이 가능합니다.

---

In [ ]:
PROMPT = """You are an Assistant that creates prompts for generate background image by image generator model. The image that Human wants is written in <keyword>.
            Write a images three creation prompts keeping it to 300 characters or less to maintain the style of a given image. Use this fomat without further explanation:
            <prompt>image prompt</prompt>

            <keyword>
            {keyword}
            </keyword>
            """

def get_mm_prompt(keyword: str):
    return PromptTemplate(
                template=PROMPT,
                input_variables=["keyword"]
            ).format(keyword=keyword)

def get_prompt_by_mm_llm(prompt: str, image: str) -> List[str]:
    claude = BedrockClaude()
    res = claude.invoke_llm_response(text=prompt, image=image)
    return extract_format(res)

In [ ]:
input_image = encode_image_base64_from_file(f"./sample/house.jpg", format="JPEG")
display_image(input_image)

prompt = get_mm_prompt(keyword=keyword)
image_prompts = get_prompt_by_mm_llm(prompt=prompt, image=input_image)
print(image_prompts)

In [ ]:
for prompt in image_prompts:
    print(prompt)
        
    # titan g1 v2
    body = image_params.text_to_image(text=f"{prompt}")
    img = titanImageGen.generate_image(body)[0]
    
    display_image(img)
    save_base64_image(img, f"{OUTPUT_PATH}/gen_titan_mmllm_{idx+1}.png")

    # stable diffusion
    img = invoke_sd_model(MODEL_STABLE_IMAGE_CORE, {
        "prompt": prompt,
        "aspect_ratio": aspect_ratio,
        "seed": seed,
        "output_format": output_format
    })
    display_image(img)
    save_base64_image(img, f"{OUTPUT_PATH}/gen_sd_mmllm_{idx+1}.png")